In [1]:
#Importing Libs
import numpy as np
import pandas as pd
import urllib.request
import requests


In [2]:
from geopy.geocoders import Nominatim # importing lib to convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # installing 
import folium # importing lib for map rendering 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
from pandas.io.json import json_normalize # lib to transform a JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests # library to handle requests
import geopy.geocoders

In [4]:
census_url = 'https://raw.githubusercontent.com/Suwhoop/coursera_capstone/master/oc_census_data.csv'
zip_url = 'https://raw.githubusercontent.com/Suwhoop/coursera_capstone/master/oc_zips.csv'

In [5]:
cendf = pd.read_csv(census_url, error_bad_lines=False)

In [6]:
cendf

,Measure,Tustin,Irvine,Newport Beach
0,"Population estimates, July 1, 2019, (V2019)","79,348","287,401","84,534"
1,"Population estimates base, April 1, 2010, (V2019)","75,317","212,107","85,211"
2,"Population, percent change - April 1, 2010 (es...",5.40%,35.50%,-0.80%
3,"Population, Census, April 1, 2010","75,540","212,375","85,186"
4,"Persons under 5 years, percent",7.60%,6.40%,3.90%
5,"Persons under 18 years, percent",25.10%,22.40%,17.40%
6,"Persons 65 years and over, percent",9.90%,9.90%,22.70%
7,"Female persons, percent",50.50%,51.20%,51.00%
8,"White alone, percent",46.00%,47.60%,85.30%
9,"Black or African American alone, percent(a)",2.70%,1.90%,0.80%


In [7]:
zipdf = pd.read_csv(zip_url, error_bad_lines=False)
zipdf.head()

,Zip Code,City,County
0,90620,Buena Park,Orange County
1,90621,Buena Park,Orange County
2,90622,Buena Park,Orange County
3,90623,Buena Park,Orange County
4,90623,La Palma,Orange County


In [8]:
loglonurl = 'https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B'
loglondf = pd.read_csv(loglonurl, sep=';', error_bad_lines=False)
loglondf.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483,-94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351,-92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847,-94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337,-85.89754"
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129,-91.98027"


In [9]:
zipdf.rename(columns={'Zip Code': 'Zip'}, inplace=True)
zipdf.head()

,Zip,City,County
0,90620,Buena Park,Orange County
1,90621,Buena Park,Orange County
2,90622,Buena Park,Orange County
3,90623,Buena Park,Orange County
4,90623,La Palma,Orange County


In [10]:
locationdf = zipdf.merge(loglondf, on=['Zip'])
locationdf.head()

,Zip,City_x,County,City_y,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,90620,Buena Park,Orange County,Buena Park,CA,33.841014,-118.009690,-8,1,"33.841014,-118.00969"
1,90621,Buena Park,Orange County,Buena Park,CA,33.875714,-117.994040,-8,1,"33.875714,-117.99404"
2,90622,Buena Park,Orange County,Buena Park,CA,33.640302,-117.769442,-8,1,"33.640302,-117.769442"
3,90623,Buena Park,Orange County,La Palma,CA,33.849327,-118.043950,-8,1,"33.849327,-118.04395"
4,90623,La Palma,Orange County,La Palma,CA,33.849327,-118.043950,-8,1,"33.849327,-118.04395"


In [12]:
NPlocationdf = locationdf['City_x'].isin(["Newport Beach", "Tustin", "Irvine"])
NPlocationdf

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18      True
19      True
20      True
21     False
22      True
23     False
24     False
25     False
26     False
27      True
28      True
29     False
       ...  
143    False
144    False
145    False
146    False
147    False
148    False
149    False
150    False
151    False
152    False
153    False
154    False
155    False
156    False
157    False
158    False
159    False
160    False
161    False
162    False
163    False
164    False
165    False
166    False
167    False
168    False
169    False
170    False
171    False
172    False
Name: City_x, Length: 173, dtype: bool

In [13]:
df = locationdf[NPlocationdf]

In [14]:
df

,Zip,City_x,County,City_y,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
18,92602,Irvine,Orange County,Irvine,CA,33.732970,-117.769320,-8,1,"33.73297,-117.76932"
19,92603,Irvine,Orange County,Irvine,CA,33.648871,-117.764637,-8,1,"33.648871,-117.764637"
20,92604,Irvine,Orange County,Irvine,CA,33.687620,-117.788520,-8,1,"33.68762,-117.78852"
22,92606,Irvine,Orange County,Irvine,CA,33.695576,-117.805880,-8,1,"33.695576,-117.80588"
27,92612,Irvine,Orange County,Irvine,CA,33.650813,-117.817710,-8,1,"33.650813,-117.81771"
28,92614,Irvine,Orange County,Irvine,CA,33.680408,-117.825920,-8,1,"33.680408,-117.82592"
30,92616,Irvine,Orange County,Irvine,CA,33.640302,-117.769442,-8,1,"33.640302,-117.769442"
31,92618,Irvine,Orange County,Irvine,CA,33.659639,-117.739480,-8,1,"33.659639,-117.73948"
32,92619,Irvine,Orange County,Irvine,CA,33.669850,-117.765939,-8,1,"33.66985,-117.765939"
33,92620,Irvine,Orange County,Irvine,CA,33.714889,-117.763300,-8,1,"33.714889,-117.7633"


In [15]:
CLIENT_ID = '0CIDC01CDR3LIBZJUZMYZKJTXCBK3VJELI31TGJ1LME3X324' # your Foursquare ID
CLIENT_SECRET = 'UCEJXIBAVRBC4OW0HXFCOXL3CRT3Z4ODIE4DHBNQJAVOUAQY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0CIDC01CDR3LIBZJUZMYZKJTXCBK3VJELI31TGJ1LME3X324
CLIENT_SECRET:UCEJXIBAVRBC4OW0HXFCOXL3CRT3Z4ODIE4DHBNQJAVOUAQY


In [94]:
irvine_neighborhood_latitude = df.loc[18, 'Latitude'] # neighborhood latitude value
irvine_neighborhood_longitude = df.loc[18, 'Longitude'] # neighborhood longitude value
newport_neighborhood_latitude = df.loc[69, 'Latitude'] # neighborhood longitude value
newport_neighborhood_longitude = df.loc[69, 'Longitude'] # neighborhood longitude value



radius = 5000
search_query = 'Venues'
LIMIT = 10000

In [95]:
irv_fs_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    irvine_neighborhood_latitude, 
    irvine_neighborhood_longitude, 
    radius, 
    LIMIT)

np_fs_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    newport_neighborhood_latitude, 
    newport_neighborhood_longitude, 
    radius, 
    LIMIT)

In [96]:
irv_results = requests.get(irv_fs_url).json()
np_results = requests.get(np_fs_url).json()

In [97]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [98]:
irv_venues = irv_results['response']['groups'][0]['items']
np_venues = np_results['response']['groups'][0]['items']

    
irv_nearby_venues = json_normalize(irv_venues) # flatten JSON
np_nearby_venues = json_normalize(np_venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
irv_nearby_venues =irv_nearby_venues.loc[:, filtered_columns]
np_nearby_venues =np_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
irv_nearby_venues['venue.categories'] = irv_nearby_venues.apply(get_category_type, axis=1)
np_nearby_venues['venue.categories'] = np_nearby_venues.apply(get_category_type, axis=1)

# clean columns
irv_nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
np_nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


In [106]:
print(np_nearby_venues.head())
print(irv_nearby_venues.head())

                                     name      categories        lat  \
0                       Total Wine & More    Liquor Store  33.638041   
1                          Back Bay Trail           Trail  33.647233   
2                        Newport Back Bay  Scenic Lookout  33.646921   
3                               Starbucks     Coffee Shop  33.638010   
4  Upper Newport Bay - Ecological Reserve  Scenic Lookout  33.640526   

          lng  
0 -117.862596  
1 -117.867913  
2 -117.885704  
3 -117.863214  
4 -117.886133  
                                    name            categories        lat  \
0                             McDonald's  Fast Food Restaurant  33.734215   
1                 Tustin Ranch Golf Club           Golf Course  33.737681   
2                 ChaCha's Latin Kitchen    Mexican Restaurant  33.729424   
3  Massage Envy - Irvine Northpark Plaza                   Spa  33.723234   
4                    Peters Canyon Trail                 Trail  33.726452   

        

In [107]:
def getIVNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    irv_nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    irv_nearby_venues.columns = ['City', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(irv_nearby_venues)

In [108]:
Irvine_venues = getIVNearbyVenues(names=df['City_x'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Newport Beach
Newport Beach
Newport Beach
Newport Beach
Newport Beach
Newport Beach
Newport Beach
Tustin
Tustin
Tustin


In [109]:
print(Irvine_venues.shape)
Irvine_venues.head()

(333, 7)


,City,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Irvine,33.73297,-117.76932,Spring Valley Fountain,33.733280,-117.769796,Park
1,Irvine,33.73297,-117.76932,Spring Valley Pool,33.734891,-117.768062,Pool
2,Irvine,33.73297,-117.76932,"Solid Solutions Construction Services, Inc.",33.732741,-117.766637,Construction & Landscaping
3,Irvine,33.73297,-117.76932,Santa Barbara Gardens Pool,33.731050,-117.771957,Pool
4,Irvine,33.73297,-117.76932,Forest Glen Pool,33.729481,-117.769310,Pool


In [110]:
newIV = Irvine_venues['City'].isin(["Irvine"])
newIV.head()

0    True
1    True
2    True
3    True
4    True
Name: City, dtype: bool

In [111]:
def getNPNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    np_nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    np_nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(np_nearby_venues)

In [112]:
Newport_venues = getNPNearbyVenues(names=df['City_x'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Irvine
Newport Beach
Newport Beach
Newport Beach
Newport Beach
Newport Beach
Newport Beach
Newport Beach
Tustin
Tustin
Tustin


In [113]:
print(Newport_venues.shape)
Newport_venues.head()

(333, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Irvine,33.73297,-117.76932,Spring Valley Fountain,33.733280,-117.769796,Park
1,Irvine,33.73297,-117.76932,Spring Valley Pool,33.734891,-117.768062,Pool
2,Irvine,33.73297,-117.76932,"Solid Solutions Construction Services, Inc.",33.732741,-117.766637,Construction & Landscaping
3,Irvine,33.73297,-117.76932,Santa Barbara Gardens Pool,33.731050,-117.771957,Pool
4,Irvine,33.73297,-117.76932,Forest Glen Pool,33.729481,-117.769310,Pool


In [114]:
newNP = Newport_venues['City'].isin(["Newport Beach"])

In [115]:
Irvine_venues = Irvine_venues[newIV]
Newport_venues = Newport_venues[newNP]

In [116]:
Irvine_venues.groupby('Venue Category').count()

,City,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,1,1,1,1,1,1
American Restaurant,1,1,1,1,1,1
Asian Restaurant,2,2,2,2,2,2
Bakery,5,5,5,5,5,5
Bank,1,1,1,1,1,1
Baseball Stadium,1,1,1,1,1,1
Big Box Store,1,1,1,1,1,1
Brewery,1,1,1,1,1,1
Bubble Tea Shop,1,1,1,1,1,1


In [117]:
Newport_venues.groupby('Venue Category').count()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
American Restaurant,4,4,4,4,4,4
Art Gallery,1,1,1,1,1,1
Arts & Crafts Store,1,1,1,1,1,1
Asian Restaurant,1,1,1,1,1,1
BBQ Joint,1,1,1,1,1,1
Bakery,1,1,1,1,1,1
Bar,3,3,3,3,3,3
Beach,8,8,8,8,8,8
Boat or Ferry,1,1,1,1,1,1


In [125]:
NPKBBQ = Newport_venues['Venue Category'].isin(["Korean Restaurant"])
IVBBQ = Irvine_venues['Venue Category'].isin(["Korean Restaurant"])
NPBAR = Newport_venues['Venue Category'].isin(["Bar", "Wine Bar", "Sports Bar", "Pub" , "Dive Bar", "Lounge"])
IVBAR = Irvine_venues['Venue Category'].isin(["Bar", "Wine Bar", "Sports Bar", "Pub" , "Dive Bar", "Lounge"])

In [126]:
NPKBBQ = Newport_venues[NPKBBQ]
IVBBQ = Irvine_venues[IVBBQ]
NPBAR = Newport_venues[NPBAR]
IVBAR = Irvine_venues[IVBAR]

In [120]:
IVBBQ

,City,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
51,Irvine,33.669850,-117.765939,The Flame Broiler,33.668729,-117.764725,Korean Restaurant
67,Irvine,33.714889,-117.763300,DooRe Korean Restaurant,33.712518,-117.761465,Korean Restaurant
71,Irvine,33.714889,-117.763300,Cho Dang Tofu & Korean B.B.Q.,33.713013,-117.761197,Korean Restaurant
74,Irvine,33.714889,-117.763300,Hako,33.712380,-117.761330,Korean Restaurant
95,Irvine,33.686519,-117.830788,Urban Seoul,33.688252,-117.833205,Korean Restaurant
101,Irvine,33.686519,-117.830788,BCD Tofu House,33.688593,-117.833606,Korean Restaurant
102,Irvine,33.686519,-117.830788,Yigah Restaurant,33.688346,-117.833068,Korean Restaurant
107,Irvine,33.686519,-117.830788,BBQ Chicken,33.688369,-117.833293,Korean Restaurant
117,Irvine,33.686519,-117.830788,Ktown - Food Court,33.689362,-117.834427,Korean Restaurant


In [121]:
NPKBBQ

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [127]:
map_irvine = folium.Map(location=[33.669850, -117.765939], zoom_start=12)

# add markers to map
for lat, lng, venue in zip(IVBBQ['Venue Latitude'], IVBBQ['Venue Longitude'], IVBBQ['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_irvine)  
    
# add markers to map
for lat, lng, venue in zip(NPKBBQ['Venue Latitude'], NPKBBQ['Venue Longitude'], NPKBBQ['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_irvine) 
    
# add markers to map
for lat, lng, venue in zip(NPBAR['Venue Latitude'], NPBAR['Venue Longitude'], NPBAR['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_irvine) 
    
# add markers to map
for lat, lng, venue in zip(IVBAR['Venue Latitude'], IVBAR['Venue Longitude'], IVBAR['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_irvine) 
    
map_irvine